In [6]:
from abc import ABC, abstractmethod
from numerapi import CryptoAPI
import pandas as pd
import numpy as np
from datetime import datetime, timezone

import json
import os
import requests
from websocket import create_connection

import eth_account
from eth_account.signers.local import LocalAccount

from hyperliquid.exchange import Exchange
from hyperliquid.info import Info
from hyperliquid.utils.constants import MAINNET_API_URL, TESTNET_API_URL

from oracle_interfaces import OracleInterface
from dex_interfaces import DEXInterface

# Oracle Implementation
class NumeraiTBNOracle(OracleInterface):
    def __init__(self):
        self.api = CryptoAPI()
        self.api.download_dataset(
        	"crypto/v1.0/historical_meta_models.csv",
        	"historical_meta_models.csv"
        )
        
        #load historical MM
        self.mm = pd.read_csv('historical_meta_models.csv')
        self.mm['date'] = pd.to_datetime( self.mm['date'] )

        self.tb = 5
        
    def fetch_portfolio_weights(self, timestamp: str, tradable_universe: list) -> dict:
        print(f"[Oracle] Computing portfolio weights for timestamp {timestamp}.")
        portfolio_date = sorted( self.mm['date'].unique()[ self.mm['date'].dt.date.unique() <= pd.to_datetime(timestamp).date() ] )[-1]
        print(f"[Oracle] Portfolio Date: {portfolio_date}")
        x = self.mm.loc[ self.mm['date'] == portfolio_date, ['symbol','meta_model'] ]

        #only keep tradable universe
        x = x.loc[ x['symbol'].isin( tradable_universe ) ]
        
        x['w'] = 0
        x.loc[  x['meta_model'] >= x['meta_model'].sort_values().iloc[-self.tb], 'w' ] = 1
        x.loc[  x['meta_model'] < x['meta_model'].sort_values().iloc[self.tb], 'w' ] = -1
        # x['w'] = x['w'] - x['w'].mean()
        print( "[Oracle] Portfolio: ", x.loc[ x['w'].abs() > 0 ] )
        x['w'] = x['w'] / x['w'].abs().sum()
        
        weights = dict(zip(x["symbol"], x["w"]))
        return weights

    def validate_weights(self, weights: dict) -> bool:
        # Ensure weights sum to 1
        valid = (np.abs( np.array( [ weights[k] for k in weights.keys() ] ) ).sum() - 1.0) < 1e-6
        if not valid:
            print("[Oracle] Weights validation failed.")
        return valid

# HyperLiquid DEX Implementation
class HyperLiquidDEX(DEXInterface):
    def __init__(self, which_net="testnet"):
        if which_net == "testnet":
            self.base_url = TESTNET_API_URL
        elif which_net == "mainnet":
            self.base_url = MAINNET_API_URL
            
        self.pwd = ""
        self.client = None
        self.universe = None
        self.positions = None
        self.set_universe()
        self.address, self.info, self.exchange, self.net_liq = self.setup()
    
    def setup(self, skip_ws=False):
        config_path = os.path.join(os.path.dirname(self.pwd), "hyperliquid_config.json")
        with open(config_path) as f:
            config = json.load(f)
        account: LocalAccount = eth_account.Account.from_key(config["secret_key"])
        address = config["account_address"]
        if address == "":
            address = account.address
        print("[DEX] Running with account address:", address)
        if address != account.address:
            print("[DEX] Running with agent address:", account.address)
        info = Info(self.base_url, skip_ws)
        user_state = info.user_state(address)
        spot_user_state = info.spot_user_state(address)
        margin_summary = user_state["marginSummary"]
        if float(margin_summary["accountValue"]) == 0 and len(spot_user_state["balances"]) == 0:
            print("[DEX] Not running because the provided account has no equity.")
            url = info.base_url.split(".", 1)[1]
            error_string = f"No accountValue:\nIf you think this is a mistake, make sure that {address} has a balance on {url}.\nIf address shown is your API wallet address, update the config to specify the address of your account, not the address of the API wallet."
            raise Exception(error_string)
        exchange = Exchange(account, self.base_url, account_address=address)
        return address, info, exchange, float(margin_summary["accountValue"])

    def get_universe(self) -> list:
        """Get the tradable assets from the DEX."""
        return self.universe.index.to_list()

    def set_universe(self):
        if self.base_url == MAINNET_API_URL:
            url = "https://api.hyperliquid.xyz/info"
        else:
            url = "https://api.hyperliquid-testnet.xyz/info"
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "type": "meta"
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            self.universe = pd.DataFrame( response.json()['universe'] ).set_index('name')
            print("[DEX] Universe Set Successfully.")
        else:
            print(f"[DEX] Request failed with status code {response.status_code}")
            print(response.text)  # Print the error message if any

    def set_portfolio_weights(self, weights: dict):
        """Fetch the current position for the given symbol."""
        self.address, self.info, self.exchange, self.net_liq = self.setup() #update info
        self.cancel_open_orders() #cancel open orders
        self.update_current_positions()

        #set each portfolio weight, 1 at a time
        for symbol in weights.keys():
            dollar_weight = weights[symbol] * self.net_liq
            if symbol in self.positions.keys():
                dollar_weight = dollar_weight - self.positions[symbol]
            if dollar_weight > 0:
                print(f'[DEX] setting weight {symbol}: {dollar_weight}')
                order_result = self.add_new_order( symbol, dollar_weight )

        for symbol in self.positions.keys():
            if symbol not in weights.keys():
                dollar_weight = - self.positions[symbol]
                print(f'[DEX] setting weight (closing) {symbol}')
                order_result = self.add_new_order( symbol, dollar_weight )

    def update_current_positions(self):
        # Get the user state and print out position information
        user_state = self.info.user_state(self.address)
        positions = {}
        for position in user_state["assetPositions"]:
            pos = position["position"]
            positions[ pos['coin'] ] = float( pos['positionValue'] ) * np.sign( float( pos['szi'] ) )
            
        if len(positions) > 0:
            print("[DEX] Getting Positions.")
            for k in positions.keys():
                print(f"[DEX] {k}: {positions[k]}")
        else:
            print("[DEX] No Open Positions")
        self.positions = positions

    def get_bid_ask(self, symbol: str ) -> (float, float):
        # WebSocket URI
        if self.base_url == MAINNET_API_URL:
            uri = "wss://api.hyperliquid.xyz/ws"
        else:
            uri = "wss://api.hyperliquid-testnet.xyz/ws"
        
        # Payload
        payload = {
            "method": "post",
            "id": 123,
            "request": {
                "type": "info",
                "payload": {
                    "type": "l2Book",
                    "coin": symbol,
                    "mantissa": None
                }
            }
        }
    
        best_bid, best_ask = None, None
        
        # Connect to the WebSocket server
        try:
            ws = create_connection(uri)
            print(f"[DEX] Connected to WebSocket getting Bid/Ask for {symbol}.")
        
            # Send the payload
            ws.send(json.dumps(payload))
            print(f"Sent: {json.dumps(payload)}")
        
            # Receive response
            response = ws.recv()
    
            resp = json.loads( response )
            levels = resp['data']['response']['payload']['data']['levels']
            best_bid = float( levels[0][0]['px'] )
            best_ask = float( levels[1][0]['px'] )
            
            # Close connection
            ws.close()
            print("[DEX] Connection closed")
        except Exception as e:
            print(f"Error: {e}")
    
        return best_bid, best_ask

    def get_open_orders(self, symbol: str) -> list:
        pass

    def cancel_open_orders(self) -> bool:
        open_orders = self.info.open_orders(self.address)
        for open_order in open_orders:
            print(f"[DEX] Cancelling order {open_order}")
            self.exchange.cancel(open_order["coin"], open_order["oid"])
        return True

    def add_new_order(self, symbol: str, dollar_weight: float) -> bool:
        if abs( dollar_weight ) < 12:
            print('[DEX] Dollar Weight too small, not sending order')
            return False
            
        best_bid, best_ask = self.get_bid_ask( symbol )
        if dollar_weight > 0:
            size = round( dollar_weight / best_ask, self.universe.loc[symbol, 'szDecimals'] )
            order_result = self.exchange.order(symbol, True, size, best_ask, {"limit": {"tif": "Gtc"}})
        else:
            size = round( abs( dollar_weight ) / best_bid, self.universe.loc[symbol, 'szDecimals'] )
            order_result = self.exchange.order(symbol, False, size, best_bid, {"limit": {"tif": "Gtc"}})
        
        # Query the order status by oid
        if order_result["status"] == "ok":
            return True
        else:
            print("[DEX] Problem with order.")
            print(order_result)
            return False

# Portfolio Manager
class PortfolioManager:
    def __init__(self, oracle: OracleInterface, dex: DEXInterface):
        """Initialize the Portfolio Manager with an Oracle implementation."""
        self.oracle = oracle
        self.dex = dex

    def manage_portfolio(self, timestamp: str):
        """Fetch weights from the Oracle and print them."""
        print(f"[PortfolioManager] Requesting portfolio weights for {timestamp}.")
        tradable_universe = dex.get_universe()
        # tradable_universe = ['BTC','ETH','AAVE','LDO','SUI','WLD','GOAT','EIGEN','AVAX','ENS']
        weights = self.oracle.fetch_portfolio_weights(
            timestamp,
            tradable_universe
        )

        if not self.oracle.validate_weights(weights):
            raise ValueError("Invalid portfolio weights received from Oracle.")

        print(f"[PortfolioManager] Portfolio weights: {weights}")

        dex.set_portfolio_weights( weights )




In [8]:
# Instantiate Oracle and DEX interfaces
oracle = NumeraiTBNOracle()
dex = HyperLiquidDEX("testnet")

# Instantiate Portfolio Manager with Oracle
portfolio_manager = PortfolioManager(oracle, dex)

# Perform portfolio management
portfolio_manager.manage_portfolio(timestamp=datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"))

[PortfolioManager] Requesting portfolio weights for 2024-12-16T16:57:54Z.
[Oracle] Computing portfolio weights for timestamp 2024-12-16T16:57:54Z.
[Oracle] Portfolio Date: 2024-12-13 00:00:00
[Oracle] Portfolio:        symbol  meta_model  w
79000    BTC       0.993  1
79001    ETH       0.995  1
79005    BNB       0.999  1
79012    TON       0.969  1
79015    XLM       0.127 -1
79016   HBAR       0.075 -1
79049    INJ       0.985  1
79057   SAND       0.153 -1
79070   IOTA       0.117 -1
79126  SUSHI       0.119 -1
[PortfolioManager] Portfolio weights: {'BTC': 0.1, 'ETH': 0.1, 'SOL': 0.0, 'BNB': 0.1, 'ADA': 0.0, 'AVAX': 0.0, 'TON': 0.1, 'SUI': 0.0, 'XLM': -0.1, 'HBAR': -0.1, 'NEAR': 0.0, 'APT': 0.0, 'ICP': 0.0, 'AAVE': 0.0, 'POL': 0.0, 'ETC': 0.0, 'RENDER': 0.0, 'VET': 0.0, 'FET': 0.0, 'TAO': 0.0, 'ARB': 0.0, 'FIL': 0.0, 'KAS': 0.0, 'FTM': 0.0, 'ALGO': 0.0, 'ATOM': 0.0, 'STX': 0.0, 'IMX': 0.0, 'TIA': 0.0, 'OP': 0.0, 'INJ': 0.1, 'WLD': 0.0, 'RUNE': 0.0, 'SAND': -0.1, 'GALA': 0.0, 'MKR':

2024-12-16 11:57:54,884 INFO websocket: Websocket connected


[DEX] Getting Positions.
[DEX] BTC: 39.58741
[DEX] ETH: 40.40424
[DEX] BNB: 40.30768
[DEX] INJ: 40.0316
[DEX] TON: 39.59735
[DEX] SUSHI: -43.68966
[DEX] HBAR: -39.33544
[DEX] XLM: -38.8233
[DEX] SAND: -43.2816
[DEX] IOTA: -39.80208
[DEX] setting weight BTC: 0.1327481000000006
[DEX] Dollar Weight too small, not sending order
[DEX] setting weight TON: 0.12280810000000031
[DEX] Dollar Weight too small, not sending order
[DEX] setting weight SAND: 3.5614418999999984
[DEX] Dollar Weight too small, not sending order
[DEX] setting weight IOTA: 0.08192189999999755
[DEX] Dollar Weight too small, not sending order
[DEX] setting weight SUSHI: 3.9695019000000045
[DEX] Dollar Weight too small, not sending order
